In [ ]:
#pip install keras_tuner

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import numpy as np
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from pandas import read_csv
import math

In [ ]:
OUTPUT_PATH = "output"
INPUT_SHAPE = (13, 21, 1)
NUM_CLASSES = 1
EPOCHS = 100
BS = 1024
EARLY_STOPPING_PATIENCE = 5

In [ ]:
df1 = pd.read_csv('CATrecon.csv')
df2 = pd.read_csv('CATlabels.csv')

X = df1.values
y = df2.values

n = 899707

X = np.reshape(X, (n,13,21,1))

df2.shape()

In [ ]:
df1.shape

In [ ]:
df2.drop('x-entry', axis=1, inplace=True)
df2.drop('y-entry', axis=1, inplace=True)
df2.drop('z-entry', axis=1, inplace=True)
df2.drop('n_x', axis=1, inplace=True)
df2.drop('n_y', axis=1, inplace=True)
df2.drop('n_z', axis=1, inplace=True)
df2.drop('number_eh_pairs', axis=1, inplace=True)
df2.drop('cotAlpha', axis=1, inplace=True)

print(df2.head())
y = df2.values

In [ ]:
def build_model(hp):
    model = Sequential()
    inputShape = INPUT_SHAPE
    chanDim = -1
    model.add(Conv2D(hp.Int("conv_1", min_value=64, max_value=256, step=32), (3, 3), strides=(2, 2), 
                        input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    for i in range(hp.Int('n_layers', 3, 5)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'conv_{i}_units', min_value=64, max_value=256, step=32), 
                         (3, 3), strides=(2, 2), padding="same" ))
        model.add(Activation('relu'))
    
    model.add(Flatten())
    model.add(Dense(hp.Int("dense_units", min_value=64, max_value=512, step=256)))
    model.add(Activation("relu"))
    model.add(Dropout(0.05))
 
    model.add(Dense(NUM_CLASSES))
    model.add(Activation("linear"))
    
    model.compile(optimizer="adam",
        loss="Huber",
        metrics=["mean_squared_error"])
    
    return model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 0)
print(X.shape, y.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#scale input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

es = EarlyStopping(
	monitor="mean_squared_error",
	patience=EARLY_STOPPING_PATIENCE,
	restore_best_weights=True)

tuner = kt.BayesianOptimization(build_model, objective="mean_squared_error", 
                                max_trials=50, 
                                seed=32, directory=OUTPUT_PATH, project_name='bayes')

print("[INFO] performing hyperparameter search...")
tuner.search(
  x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=BS, callbacks=[es], epochs=EPOCHS
	)

bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
tuner.get_best_models()[0].summary()

